### Color to Gray

In [ ]:
import cv2
import streamlit as st

def video_to_grayscale(vid_path):
    # open the video
    cap = cv2.VideoCapture(vid_path)

    # create a writer object to save the grayscale video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('grayscale_video.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), 
                          (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))), 
                          isColor=False)

    # loop over frames in the video
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # write grayscale frame to writer object
        out.write(gray)

    # release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# streamlit app
st.title('Video to Grayscale')
vid_file = st.file_uploader('Upload a video file', type=['mp4'])

if vid_file is not None:
    # display uploaded video
    st.video(vid_file)

    # convert video to grayscale and save
    video_to_grayscale(vid_file.name)

    # display link to download grayscale video
    st.markdown('Download grayscale video [here](./grayscale_video.mp4)')

### YuNET face detection

In [ ]:
cam = cv2.VideoCapture(0)
detector = cv2.FaceDetectorYN.create("yunet_22mar.onnx", "", (320, 320))
i=0
j=0
n_images=50
k=int(n_images/10)
while True:
    ret, collection_frame = cam.read()
    if not ret:
        break

    height, width, _ = collection_frame.shape
    detector.setInputSize((width, height))
    _, faces = detector.detect(collection_frame)
    if faces is not None:
        for face in faces:
            if i%10==0:
                j+=1
            x,y,w,h = list(map(int, face[:4]))
            cv2.putText(collection_frame, f'Images Captured: {j}/{k}',(30,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 127, 0),1,cv2.LINE_AA)
            cv2.rectangle(collection_frame, (x,y), (x+w,y+h), (255,0,0), 2)   
    
    i+=1
    cv2.imshow('img', collection_frame)
    if j==n_images:
        break
    if cv2.waitKey(1)==27:
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
cam = cv2.VideoCapture(0)
detector = cv2.FaceDetectorYN.create("yunet_22mar.onnx", "", (320, 320))
i=0
n_images=500
id=input('id: ')
path='dataset/'+id
if not os.path.isdir(path):
    os.makedirs(path)

while True:
    ret, collection_frame = cam.read()
    if not ret:
        break

    height, width, _ = collection_frame.shape
    detector.setInputSize((width, height))
    _, faces = detector.detect(collection_frame)
    if faces is not None:
        for face in faces:
            x,y,w,h = list(map(int, face[:4]))
            cv2.putText(collection_frame, f'Images Captured: {i}/{n_images}',(30,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 127, 0),1,cv2.LINE_AA)
            cv2.imwrite(path + '/' +str(i) + ".jpg", collection_frame)
            cv2.rectangle(collection_frame, (x,y), (x+w,y+h), (255,0,0), 2)   
            i+=1

    cv2.imshow('img', collection_frame)
    if i==n_images:
        break
    if cv2.waitKey(1)==27:
        break

cam.release()
cv2.destroyAllWindows()

In [ ]:
# with tf.device('/GPU:0'):
image_width = 224
image_height = 224

detector = cv2.FaceDetectorYN.create("yunet_22mar.onnx", "", (320, 320))
model = load_model('emp_face_vgg_model.h5', compile=False)
with open("emp-face-labels.pickle", 'rb') as f:
    og_labels = pickle.load(f)
    labels = {key:value for key,value in og_labels.items()}

stream = cv2.VideoCapture(0)
while(True):
    (grabbed, frame) = stream.read()
    height, width, _ = frame.shape
    detector.setInputSize((width, height))
    _, faces = detector.detect(frame)
    if faces is not None:    
        for face in faces:
            x,y,w,h = list(map(int, face[:4]))

            color = (255, 127, 0)
            stroke = 2
            cv2.rectangle(frame, (x,y), (x+w,y+h), color, stroke)

            roi_rgb = frame[y:y+h, x:x+w]
            size = (image_width, image_height)
            resized_image = cv2.resize(roi_rgb, size)
            image_array = np.array(resized_image, "uint8")
            img = image_array.reshape(1,image_width,image_height,3) 
            img = img.astype('float32')
            img /= 255
            predicted_prob = model.predict(img)

            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[predicted_prob[0].argmax()]
            color = (255, 0, 255)
            stroke = 2
            cv2.putText(frame, f'({name})', (x,y-8), font, 1, color, stroke, cv2.LINE_AA)


    cv2.imshow("Image", frame)
    if cv2.waitKey(1)==27:
        break

stream.release()
cv2.destroyAllWindows()

### MTCNN face detection

In [ ]:
from mtcnn import MTCNN
import cv2, os

username = 'user'
path = 'faces/'+username
count=0
n_of_images=100
while os.path.isdir(path):
    count+=1
    path='faces/'+username+str(count)  
os.makedirs(path)

cap = cv2.VideoCapture(0)
i,j = 0,0
while 1:
    _,frame = cap.read()
    detector = MTCNN()
    r = detector.detect_faces(frame)
    if r==[]:
        continue
    faces = r[0]['box']
    (x,y,w,h) = faces
    cv2.putText(frame,f'Images Captured: {i}/{n_of_images}',(30,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 0, 20),1,cv2.LINE_AA)
    name = str(i)+'.jpg'
#     cv2.imwrite(path+'/'+name,frame[y:y+h,x:x+w])
    cv2.imwrite(path+'/'+name,frame)
    cv2.rectangle(frame,(x, y), (x+w, y+h), (255, 0, 20), 2)
    i+=1
    if i==n_of_images:
        break
    cv2.imshow('Adding new User',frame)
    if cv2.waitKey(1)==27:
        break
cap.release()
cv2.destroyAllWindows()

### pickle

In [ ]:
import pickle

class_dictionary = train_generator.class_indices

with open('emp-face-labels.pickle',"rb") as f:
    obj = pickle.load(f)
    print(len(obj), obj)
    
    for key, value in class_dictionary.items():
        obj[value+len(obj)]=key
    print(obj)
        
with open("emp-face-labels.pickle","wb") as f:
    pickle.dump(obj, f)